In [10]:
import numpy as np
import math
import sys
import matplotlib.pyplot as plt

In [11]:
# the following are calibrated for z= 0-8 but we're going to extrapolate to larger z
# from Behroozi and Wechsler, eq 3 w/ parameters on page 10, sec 5
# https://arxiv.org/pdf/1207.6105.pdf

def a(z):
    return 1/(1+float(z))

def nu(z):
    return np.exp(-4*(a(z))**2)

def alpha(z):
    return -1.412 + .731*(a(z)-1)*nu(z)

def delta(z):
    return 3.508 + (2.608*(a(z)-1) - .043*z)*nu(z)

def gamma(z):
    return .316 + (1.319*(a(z)-1) + .279*z)*nu(z)

def log10_m_1(z):
    return  11.514 - (1.793*(a(z)-1) + .251*z)*nu(z) 

def log10_epsilon(z):
    return  -1.777 - .119*(a(z)-1) - .006*(a(z)-1)*nu(z) 

def f(x,z):
    a = -np.log10(-np.power(10,((alpha(z)*x) + 1)))
    b = delta(z)*np.power((np.log10(1+np.exp(x))),gamma(z))
    b = b/(1+np.exp(np.power(10, -x)))
    return a+b

def calc_mass(m_h,z):
    a = log10_epsilon(z) + log10_m_1(z)
    b = f(np.log10(m_h) - log10_m_1(z),z)
    c = f(np.zeros(np.size(m_h)),z)
    log10_m_s = a+b-c
    return np.power(10, log10_m_s)

In [ ]:
#J1 from https://arxiv.org/pdf/1806.07893.pdf. I'm not sure if it's safe to say M_peak = M_halo

def a(z):
    return 1/(1+float(z))

def log10_m_1(z):
    return

def alpha(z):
    return

def beta(z):
    return

def gamma(z):
    return

def delta(z):
    return

def epsilon(z):
    return



def calc_mass2(m_h, z):
    x = np.log10(m_h) - log10_m_1(z)
    a = np.log10(np.power(10, -alpha(z)*x) + np.power(10,-beta(z)*x))
    b = gamma(z)*np.exp(-0.5*np.power(x,2)/(delta(z)*2))
    log10_m_s = epsilon(z) + log10_m_1(z) - a + b

In [12]:
#Raymond's Old MM and LW data
LW=[   [16.455053237912374, 16.179178835251676, 15.909029421711196, 15.65278934221482, 15.406890894175554, 15.155088852988694, 15.0, 14.926102882624622, 14.703517587939698, 14.489467162329614, 14.28117359413203, 14.080681646810437, 13.887598630340927, 13.699397324709688, 13.518002322880372, 13.34102968593145, 13.17032733456143, 13.003640946646128, 12.84083044982699, 12.685507048036131, 12.531799729364007, 12.383297644539615, 12.239772275916854, 12.099292638197536, 11.961762799740765, 11.828736369467606, 11.698412698412698, 11.572290671360323, 11.448649321548611, 11.32741617357002, 11.21001221001221, 11.094823415578132, 10.983223487118035, 10.872254541137362, 10.76470588235294, 10.66044776119403, 10.55802126675913, 10.457378551787352, 10.359763716914689, 10.262529564140106, 10.169440411035408, 10.076650420912715, 9.986596352450011, 9.897994768962512, 9.81081081081081, 9.726161106939827, 9.642826734780758, 9.560777273207309, 9.481081647626036, 9.402579839800271, 9.325245224574083, 9.24905196269345, 9.175010175010174, 9.1010101010101, 9.029084344599339, 8.959167413604224, 8.889240506329113, 8.821253191907289, 8.754194303550527, 8.688983625617674, 8.6237128284092, 8.560229445506693, 8.497578117580018, 8.435742592942065, 8.374707040404987, 8.315323707498836, 8.256687957048968, 8.198785760279643, 8.141603437242892, 8.08595311648192],             [346619.71830985916, 901126.7605633803, 2460563.3802816905, 5719718.309859156, 7278873.23943662, 12374647.887323944, 28211267.605633806, 47211267.60563381, 16042253.521126762, 17225352.112676058, 36943661.97183099, 18929577.464788735, 12878873.23943662, 6828169.014084508, 239859154.9295775, 248169014.08450708, 223661971.83098593, 159014084.50704226, 94422535.21126762, 71436619.71830986, 27521126.76056338, 13309859.154929578, 359014084.5070423, 368873239.43661976, 367183098.59154934, 382957746.47887325, 428591549.2957747, 432816901.4084507, 482676056.3380282, 386760563.38028175, 433380281.6901409, 439436619.7183099, 445492957.7464789, 451549295.77464795, 453943661.971831, 214507042.25352114, 385211267.60563385, 203098591.54929578, 455211267.60563385, 417746478.87323946, 746056338.028169, 1074366197.1830988, 714929577.4647888, 622957746.4788733, 477887323.943662, 863380281.690141, 2740845070.4225354, 3387323943.661972, 4033802816.9014087, 11681690140.845072, 19464788732.394367, 7280281690.140845, 10064788732.394367, 12849295774.647888, 15633802816.90141, 10314084507.042253, 11122535211.267607, 11929577464.788733, 94028169014.08452, 93422535211.26761, 99816901408.45071, 102577464788.7324, 104056338028.16902, 107169014084.50705, 66957746478.873245, 112436619718.30988, 112929577464.78874, 112732394366.19719, 88394366197.1831, 116014084507.04227],         [346619.71830985916, 6690140.845070424, 73591549.29577465, 48211267.60563381, 46816901.40845071, 12374647.887323944, 28211267.605633806, 47211267.60563381, 16042253.521126762, 17225352.112676058, 160985915.49295777, 195492957.7464789, 195070422.5352113, 19422535.21126761, 239859154.9295775, 248169014.08450708, 223661971.83098593, 297183098.59154934, 314929577.46478873, 332816901.4084507, 347605633.8028169, 18647887.323943663, 359014084.5070423, 368873239.43661976, 367183098.59154934, 382957746.47887325, 428591549.2957747, 432816901.4084507, 4932394366.197184, 5935211267.605635, 6545070422.535212, 3690140845.0704226, 2953521126.760564, 8185915492.957747, 1220000000.0, 8738028169.014086, 2209859154.929578, 2192957746.4788733, 2177464788.7323947, 4442253521.1267605, 746056338.028169, 2204225352.112676, 2471830985.915493, 1402112676.056338, 59901408450.70423, 57830985915.492966, 6132394366.197184, 7276056338.02817, 6376056338.02817, 15591549295.774649, 25436619718.30986, 27605633802.816902, 29774647887.323948, 20647887323.943665, 37676056338.028175, 39281690140.84508, 18845070422.535213, 89098591549.29578, 94028169014.08452, 93422535211.26761, 99816901408.45071, 102577464788.7324, 104056338028.16902, 107169014084.50705, 66957746478.873245, 112436619718.30988, 112929577464.78874, 112732394366.19719, 88394366197.1831, 116014084507.04227], [ 0, 0.004203926735805602, 0.0038712659428108325, 0.00752458636736847, 0.006905077841706204, 0, 0, 0, 0, 0, 0.009849192860329007, 0.00511445011706563, 0.003609141172079399, 0.005070601542222878, 0, 0, 0, 0.010083223690860935, 0.010049009901476563, 0.006564426859976727, 0.0050605730900755925, 0.0033314261210496588, 0, 0, 0, 0, 0, 0, 0.03382256051809499, 0.026973991176685343, 0.03528983281343344, 0.030408581025755874, 0.0032562862282057535, 0.0419838647577856, 0.020546208896047465, 0.04579999344977947, 0.022491931886788715, 0.024560482487119225, 0.02523946512904181, 0.02782209553574532, 0, 0.01663614739054601, 0.019924680674983868, 0.02375244197971879, 0.0707445898991597, 0.06840407151624572, 0.02686798280481672, 0.0323710812300113, 0.03320405246351611, 0.03299796508877371, 0.03321037337941296, 0.01924586449084434, 0.024647857919098376, 0.03886461758463408, 0.03595307219139895, 0.010797309850146751, 0.02589969111785076, 0.015895263445437395, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]    ] 
LWz=LW[0]
LWm=LW[1]
LWhm=LW[2]
LWr=LW[3]

MM=[   [16.455053237912374, 16.179178835251676, 15.909029421711196, 15.65278934221482, 15.406890894175554, 15.155088852988694, 15.0, 14.926102882624622, 14.703517587939698, 14.489467162329614, 14.28117359413203, 14.080681646810437, 13.887598630340927, 13.699397324709688, 13.518002322880372, 13.34102968593145, 13.17032733456143, 13.003640946646128, 12.84083044982699, 12.685507048036131, 12.531799729364007, 12.383297644539615, 12.239772275916854, 12.099292638197536, 11.961762799740765, 11.828736369467606, 11.698412698412698, 11.572290671360323, 11.448649321548611, 11.32741617357002, 11.21001221001221, 11.094823415578132, 10.983223487118035, 10.872254541137362, 10.76470588235294, 10.66044776119403, 10.55802126675913, 10.457378551787352, 10.359763716914689, 10.262529564140106, 10.169440411035408, 10.076650420912715, 9.986596352450011, 9.897994768962512, 9.81081081081081, 9.726161106939827, 9.642826734780758, 9.560777273207309, 9.481081647626036, 9.402579839800271, 9.325245224574083, 9.24905196269345, 9.175010175010174, 9.1010101010101, 9.029084344599339, 8.959167413604224, 8.889240506329113, 8.821253191907289, 8.754194303550527, 8.688983625617674, 8.6237128284092, 8.560229445506693, 8.497578117580018, 8.435742592942065, 8.374707040404987, 8.315323707498836, 8.256687957048968, 8.198785760279643, 8.141603437242892, 8.08595311648192],             [7625352.112676057, 8839436.619718311, 7521126.760563381, 8145070.422535212, 7036619.71830986, 18887323.943661973, 80098591.54929578, 85690140.84507044, 96323943.66197184, 106619718.30985917, 120971830.9859155, 118436619.71830986, 148591549.29577467, 139436619.71830988, 165352112.67605636, 173380281.69014087, 183380281.69014087, 184647887.32394367, 200140845.07042256, 209436619.71830988, 218028169.01408452, 217464788.7323944, 96816901.40845071, 250140845.07042256, 259859154.9295775, 125859154.92957747, 117225352.11267607, 170000000.0, 64126760.563380286, 59521126.76056339, 54901408.45070423, 21197183098.591553, 21915492957.74648, 23042253521.126762, 23830985915.492958, 24901408450.704227, 25591549295.77465, 26352112676.05634, 27450704225.352116, 24014084507.042255, 25676056338.02817, 27760563380.281693, 29014084507.042255, 30028169014.08451, 30535211267.605637, 32366197183.098595, 29605633802.816902, 33028169014.08451, 30943661971.83099, 26183098591.549297, 37239436619.718315, 38802816901.408455, 35873239436.61972, 40464788732.39437, 41253521126.76057, 43056338028.16902, 44197183098.59155, 49464788732.39437, 54718309859.15494, 59985915492.95775, 66957746478.873245, 73915492957.74649, 80887323943.66197, 60394366197.183105, 91450704225.35213, 69380281690.14085, 47309859154.92958, 88563380281.69016, 68084507042.253525, 110197183098.59155],          [7625352.112676057, 8839436.619718311, 7521126.760563381, 11646478.873239437, 75323943.66197184, 82661971.83098592, 87422535.21126762, 81070422.53521128, 96323943.66197184, 106619718.30985917, 120971830.9859155, 118436619.71830986, 148591549.29577467, 139436619.71830988, 165352112.67605636, 173380281.69014087, 183380281.69014087, 184647887.32394367, 200140845.07042256, 209436619.71830988, 218028169.01408452, 217464788.7323944, 96816901.40845071, 250140845.07042256, 259859154.9295775, 125859154.92957747, 117225352.11267607, 220422535.21126762, 64126760.563380286, 19647887323.943665, 20225352112.67606, 21197183098.591553, 21915492957.74648, 23042253521.126762, 23830985915.492958, 24901408450.704227, 25591549295.77465, 26352112676.05634, 27450704225.352116, 24014084507.042255, 25676056338.02817, 27760563380.281693, 29014084507.042255, 30028169014.08451, 30535211267.605637, 32366197183.098595, 29605633802.816902, 33028169014.08451, 30943661971.83099, 26183098591.549297, 37239436619.718315, 38802816901.408455, 35873239436.61972, 40464788732.39437, 41253521126.76057, 43056338028.16902, 44197183098.59155, 49464788732.39437, 65605633802.81691, 59985915492.95775, 77816901408.45071, 73915492957.74649, 80887323943.66197, 88352112676.05635, 91450704225.35213, 93563380281.69016, 98436619718.30986, 101704225352.11269, 104957746478.87325, 110197183098.59155], [ 0, 0, 0, 0.004822789649155053, 0.007368459811927571, 0.0074099730093987425, 0.0031429444793061754, 0.005329971857335843, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.009597364221492762, 0, 0.05562099783355257, 0.05977221177102313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.015726452238187815, 0, 0.0647683178722449, 0, 0, 0.0268183034511894, 0, 0.010622678569929506, 0.03232852919636058, 0.06172105151405045, 0.02866559959254286, 0 ]   ]
MMz=MM[0]
MMm=MM[1]
MMhm=MM[2]
MMr=MM[3]